In [5]:
import numpy as np
from scipy import linalg
from scipy import special
from matplotlib import pyplot as plt

from copy import deepcopy

##### **Procedure to compute** $U_1, \dots, U_l$

**Comment** \[Input to this procedure consists of the number of points $n$, the number of zero moments $k$, and the points $x_1, \dots, x_n$. Output is the matrices $U_{j,i}$ for $j = 1, \dots, l$ and $\displaystyle i = l, \dots, \frac{n}{2^jk}$, which make up the matrices $U_1, \dots, U_l$ (note $\displaystyle l = \log_2 \left( \frac{n}{k} \right)$).\]

<h5 style="text-align: center; font-weight: bold;"> Step 1. </h5>

Compute the shifted and scaled moments matrices $M'_{1,i}$ for $\displaystyle i = 1, \dots, \frac{n}{2k}$.

In [ ]:
# In text:
# j = 1, ..., l
# i = 1, ..., n/(2^j*k)

# In this cell:
# j = 0, ..., l-1
# i = 0, ..., n/(2^j*k)-1

def mu_builder (xs, k):
    def mu (j, i):
        return ( xs[ 1+2**j*k*i ] + xs[ 2**j*k*(i+1) ] ) / 2
    
    return mu

def sigma_builder (xs, k):
    def sigma (j, i):
        return ( xs[ 2**j*k*(i+1) ] - xs[1 + 2**j*k*i] ) / 2
    
    return sigma

def s_builder (k):
    def s (i):
        return 2*k*i
    
    return s

In [ ]:
def shifted_scaled_matrices (xs, k, l):
    n = len(xs)
    assert n % (2*k) == 0
    
    mu_function = mu_builder(xs, k)
    sigma_function = sigma_builder(xs, k)
    s_function = s_builder(k)
    
    Ms = []
    for i in range(n//(2*k)): # matrices loop
        mu = mu_function(1, i)
        sigma = sigma_function(1, i)
        s = s_function(i)
        
        M = np.ones((2*k,2*k))
        for r in range(2*k): # rows loop
            for c in range(1,2*k): # entries loop
                M[r,c] = (xs[s + i] - mu) / sigma
        
        Ms.append( deepcopy(M) )
    
    return Ms

<h5 style="text-align: center; font-weight: bold;">Step 2.</h5>

Compute $U_{1,i}$ from $M'_{1,i}$ by Eq. (4.11) using Gram-Schmidt (?) orthogonalization for $i = 1,\dots, \frac{n}{2k}$.

In [ ]:
def U_1_i (M):
    return linalg.qr(M).pop(0).T

def U_1s (Ms):
    return [ [U_1_i(M) for M in Ms] ]

<h5 style="text-align: center; font-weight: bold;">Step 3.</h5>

**Comment** \[Compute $M'_{j,i}$ and $U_{j,i}$ for $j = 2, \dots, l$ and $\displaystyle i = 1, \dots, \frac{n}{2^jk}$\.]

<!-- Seja M uma matriz m x n, M^U é a matriz m/2 x n formada pelas m/2 primeiras linhas de M -->
<!-- Seja M uma matriz m x n, M^L é a matriz m/2 x n formada pelas m/2 últimas linhas de M -->

<img src="step-3_algorithm.jpg">

<!--
```
do j = 2, ..., l
    do i = 1, ..., n/(2^j*k)
        Compute U_j-1,2i-1^U M'_j-1,2i-1 and U_j-1,2i^U M'_j-1,2i.
        Compute S^1_j,i by Eq. (4.16) and S^2_j,i by Eq. (4.17);
            multiply to obtain M'_j,i by Eq. (4.13).
        Orthogonalize M'_j,i to obtain U_j,i by Eq. (4.11).
    enddo
enddo
```
-->

##### **Procedure to compute** $UTU^T$

**Comment** [Input to this procedure consists of $n, k$, the matrices $U_{j,i}$ computed above, a function to compute elements of $T$, and the chosen precision $\varepsilon$. Output is a matrix $R_l$ such that $\lVert R_l - UTU^T \rVert \lt \varepsilon \lVert T \rVert $.]

In [ ]:
def upper_half (matrix):
    # checks if there's an even number of rows
    rows = matrix.shape[0]
    assert rows % 2 == 0
    
    return matrix[:rows//2]

In [ ]:
def Us_builder (xs, k, l):
    n = len(xs)
    
    def Us (U_1, Ms):
        U = [U_1]
        
        for j in range(l):
            for i in range(n//(2**j*k)):
                pass
    
        return #
    
    return Us

In [ ]:
def S_entry (mu, sigma, i, j):
    return special.binom(j, i)*(-mu)**(j-i)/sigma**(j)

def S_builder (xs, k):
    def S (mu, sigma):
        return [ [ S_entry(mu, sigma, i, j) for j in range(2*k) ] for i in range(2*k) ]
    
    return

def S_1_builder (xs, k, mu=None, sigma=None):
    if mu is None:
        mu = mu_builder(xs, k)
    
    if sigma is None:
        sigma = sigma_builder(xs, k)
    
    S = S_builder(xs, k)
    
    def S_1 (j, i):
        # this indices (j, i) are not the entries of the matrix
        # instead those denote a matrix in a collections of matrices
        m = (mu(j, i) - mu(j-1, 2*i-1)) / sigma(j-1, 2*i-1)
        s = sigma(j, i) / sigma(j-1, 2*i-1)
        return S(m, s)
    
    return S_1

def S_2_builder (xs, k, mu=None, sigma=None):
    if mu is None:
        mu = mu_builder(xs, k)
    
    if sigma is None:
        sigma = sigma_builder(xs, k)
    
    S = S_builder(xs, k)
    
    def S_2 (j, i):
        m = (mu(j, i) - mu(j-1, 2*i)) / sigma(j-1, 2*i)
        s = sigma(j, i) / sigma(j-1, 2*i)
        return S(m, s)
    
    return S_2

<h5 style="text-align: center; font-weight: bold;">Step 4.</h5>

Compute the $k \times k$ extracts, indicated by Eq. (4.18), of the submatrices of $T$ shown in Fig. 4.4.

<img src="fig_4.4.jpg" style="width: 70%;">

<h5 style="text-align: center; font-weight: bold;">Step 5.</h5>

Extract the matrices $P''$ (Eq. (4.19)) from $U_1, U_2U_1, \dots, U_l \dots U_1$ and compute $W_0, \dots, W_{l-2}$ according to Eqs. (4.20).

<h5 style="text-align: center; font-weight: bold;">Step 6.</h5>

Compute $R_0, \dots, R_l$ by Eq. (4.21), discarding elements below a threshold $\tau$ determined by the precision $\varepsilon$ (Eq. (4.22))

##### **Procedure to compute** $UT^{-1}U^T$

**Comment** \[Input to this procedure consists of n the matrix $R_l$ which approximates $UTU^T$, and the precision $\varepsilon$. Output is a matrix $X_m$ that approximates $UT^{-1}U^T$.\]

<h5 style="text-align: center; font-weight: bold;">Step 7.</h5>

Compute the matrix $\displaystyle X_0 = \frac{R_l T R_l}{\lVert R_l T R_l \rVert}$ by direct matrix multiplication, discarding elements below a threshold $\tau$ determined by the precision $\varepsilon$ (Eq. (4.22))

<h5 style="text-align: center; font-weight: bold;">Step 8.</h5>

**Comment** \[Obtain the inverse by Schulz iteration.\]

```
do m = 0, 1, ... while ||I - X_m R_l|| > epsilon
    Compute X_m+1 = 2 X_m - X_m R_l X_m, discarding elements below threshold
enddo
```